In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
### URL
URL = "https://www.tigerdirect.com/applications/SearchTools/item-details.asp?EdpNo=1501390"

### create user agent to imitate real person
headers = {'User-Agent': 'Mozilla/5.0'}

### request access
r = requests.get(URL , headers = headers)


In [3]:
### Assign bs4 object to soup
soup = BeautifulSoup(r.content, 'html5lib')

## List price

In [13]:
### locate "List price"
table = soup.find('del' , attrs = {'aria-hidden':'true'})

### check list price
list_price = str(table.contents[0])
list_price

'$1,399.99'

## Current price

In [14]:
### locate "New price"
new_table = soup.find('p' , attrs = {'class':'final-price'}).find('span' , attrs = {'class':'sr-only'})
new_price_str = str(new_table.contents[0])

### get rid of redundant characters
current_price = ""
for char in new_price_str:
    if char.isnumeric() == True:
        current_price = current_price + char
    elif char == 'a': ### add '.' while seeing 'a'(represent of 'and')
        current_price = current_price + '.'
    elif char == '$' or char == ',': ### preserve dollar sign for next question
        current_price = current_price + char
        
### check current price
current_price



'$1,029.99'

## Regex

In [15]:
def regex_for_price(price):
    ### replace comma
    price = re.sub("," , "" , price)
    
    ### return "number.number" string
    return re.search("[0-9]+\.[0-9]+" , price)[0]
    

### list price regex
reg_list_price = regex_for_price(list_price)


### current price regex
reg_current_price = regex_for_price(current_price)


print(f'List price after regex : {reg_list_price}')
print(f'Current price after regex : {reg_current_price}')

List price after regex : 1399.99
Current price after regex : 1029.99


## Load "https://www.usnews.com/"

In [2]:
### URL
URL = "https://www.usnews.com/"

headers = {'User-Agent': 'Mozilla/5.0'}

### request access
r = requests.get(URL , headers = headers)


In [3]:
### Assign bs4 object to soup
soup = BeautifulSoup(r.content, 'html5lib')

## Find Top Stories

In [4]:
### catch all head lines
all_head_lines = soup.find('div' , attrs = {'class' : 'Box-w0dun1-0 ArmRestTopStories__Part-s0vo7p-1 erkdnc biVKSR'}).find_all('h3')

head_line_list = []

### separate headlines and store into list
for lines in all_head_lines:
    head_line_list.append(lines.text)
    
### print headlines
counter = 1
for lines in head_line_list:
    print(f"Head line counter : {lines}")
    counter = counter + 1

Head line counter : McCarthy, Biden to Talk Amid Debt Threat
Head line counter : Existing Homes Fall 1.5% in December


## Print URL of second current top story

In [5]:
top_story2_href = all_head_lines[1].find('a')['href']
top_story2_href

'https://www.usnews.com/news/economy/articles/2023-01-20/existing-homes-fall-1-5-in-december-marking-11th-month-of-declines'

## Load top story 2

In [8]:
### request access
headers = {'User-Agent': 'Mozilla/5.0'}

### request access
r = requests.get(top_story2_href , headers = headers)

In [9]:
soup = BeautifulSoup(r.content, 'html5lib')

### Header

In [10]:
header = soup.find('h1' ,attrs = {'class' : 'Heading-sc-1w5xk2o-0 iQhOvV'}).text
print(header)

Existing Homes Fall 1.5% in December, Marking 11th Month of Declines


### First 3 sentences

In [14]:
### print all sentences
temp_sentences = ''

for n in range(0 , len(sentences)):
    temp_sentences = temp_sentences + sentences[n].text

empty_str = ""
counter = 0
for char_index in range(0, len(temp_sentences) - 1):
    if temp_sentences[char_index] == '.' and temp_sentences[char_index + 1] == ' ':
        print(f"Setences {counter + 1} : ",empty_str + '.')
        print("")
        empty_str = ""
        counter = counter + 1
    elif counter >= 3:
        break
    else:
        empty_str = empty_str + temp_sentences[char_index]
    
    
print(empty_str)

Setences 1 :  Sales of existing homes slid 1.5% in December, somewhat better than expected but the 11th straight month of decline, the National Association of Realtors said on Friday.The number was better than estimates of a 3.4% drop and brings the annual rate of home sales just a hair above 4 million.

Setences 2 :   Sales are now down 34% from year-ago levels.“December was another difficult month for buyers, who continue to face limited inventory and high mortgage rates,” said NAR Chief Economist Lawrence Yun.

Setences 3 :   “However, expect sales to pick up again soon since mortgage rates have markedly declined after peaking late last year.”Prices, meanwhile, were slightly higher with the median selling price of $366,900 coming in at 2.3% above a year ago.Housing has taken it on the chin since the Federal Reserve began raising interest rates in earnest last year to bring down inflation.


